# Create People Collect 2

In [1]:
import pandas as pd

In [2]:
pc = pd.read_csv('people_collect/people_collect_2.csv')
direct_1790_matched = pd.read_csv('matches/direct_matches.csv')
baptisms = pd.read_csv('data/Baptisms.csv')
census_1790 = pd.read_csv('data/1790 Census Data Complete.csv')
census_1790['ecpp_id'] = range(1, len(census_1790) + 1)

## Insert Matched 

In [3]:
census_1790.columns

In [4]:
baptisms.columns

In [5]:
census_1790.Race

In [6]:
direct_1790_matched

In [7]:
pc.columns

In [8]:
def clean_and_reformat_race(race_value):
    if pd.isnull(race_value):
        return []
    cleaned_list = [
        item.strip().strip('.').replace('[', '').replace(']', '')
        for item in str(race_value).split(',') if item.strip()
    ]
    return cleaned_list

## Baptisms & 1790 Census

In [9]:
# access values that are matched between baptisms & 1790 census and insert into people_collect 2.0
# will need function to get parents names
# function to aggregate race
data_to_append = []

for index, row in direct_1790_matched.iterrows():
    pc.loc[index, 'last_name'] = census_1790.loc[row['ecpp_id'], 'Last']
    pc.loc[index, 'first_name'] = census_1790.loc[row['ecpp_id'], 'First']
    pc.loc[index, 'father_last_name'] = census_1790.loc[row['ecpp_id'], 'Father_Last']
    pc.loc[index, 'father_first_name'] = census_1790.loc[row['ecpp_id'], 'Father_First']
    pc.loc[index, 'mother_last_name'] = census_1790.loc[row['ecpp_id'], 'Mother_Last']
    pc.loc[index, 'mother_first_name'] = census_1790.loc[row['ecpp_id'], 'Mother_First']
    # maybe redundant?
    pc.loc[index, 'father_origin'] = baptisms.loc[row['#ID'], 'FOrigin']
    pc.loc[index, 'mother_origin'] = baptisms.loc[row['#ID'], 'FOrigin']

    pc.loc[index, 'father_military_status'] = baptisms.loc[row['#ID'], 'FMilitaryStatus']
    pc.loc[index, 'sex'] = baptisms.loc[row['#ID'], 'Sex']

    pc.loc[index, 'race_1790'] = census_1790.loc[row['ecpp_id'], 'Race']
    pc.loc[index, 'ethnicity'] = baptisms.loc[row['#ID'], 'Ethnicity']
    pc.loc[index, 'baptismal_date'] = baptisms.loc[row['#ID'], 'FmtdDate']
    pc.loc[index, 'location_ecpp_baptism'] = baptisms.loc[row['#ID'], 'DMission']
    pc.loc[index, 'location_1790_census'] = census_1790.loc[row['ecpp_id'], 'Current_Location']
    pc.loc[index, 'origin_parish_1790_census'] = census_1790.loc[row['ecpp_id'], 'Origin Parish']
    pc.loc[index, 'location_other_race'] = census_1790.loc[row['ecpp_id'], 'Location Other Race']

    # popoulate pc_id
    data_to_append.append({'pc_id': index, '1790_ecpp_id': row['ecpp_id'], 'baptisms_id': row['#ID']})

    

pc['race_1790'] = pc['race_1790'].apply(clean_and_reformat_race)
pc['ethnicity'] = pc['ethnicity'].apply(clean_and_reformat_race)
pc_id = pd.DataFrame(data_to_append)

## Baptisms & 1821 Padron

In [10]:
# values are repetitive, but ecpp_id and baptisms id are unique. This means records are duplicated
pc.to_csv('people_collect/people_collect_2.csv', index=False)

In [11]:
pc_id.to_csv('people_collect/people_collect_2_ids.csv', index=False)

In [12]:
pc_id